In [ ]:
https://medium.com/mlearning-ai/predict-sp500-stock-price-with-python-machine-learning-sentiment-analysis-a296dc276353


In [ ]:
amazon1 = './newsarticle_amazon_2019.csv'


In [3]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials


In [4]:
#get stock data from yahoo finance
stock_df = yf.download('AMZN', 
                      start='2019-01-01', 
                      end='2022-01-01', 
                      progress=False,
)
stock_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-01-02,73.260002,77.667999,73.046501,76.956497,76.956497,159662000
2019-01-03,76.000504,76.900002,74.855499,75.014000,75.014000,139512000
2019-01-04,76.500000,79.699997,75.915497,78.769501,78.769501,183652000
2019-01-07,80.115501,81.727997,79.459503,81.475502,81.475502,159864000
2019-01-08,83.234497,83.830498,80.830498,82.829002,82.829002,177628000


In [24]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 757 entries, 2019-01-02 to 2021-12-31
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       757 non-null    float64
 1   High       757 non-null    float64
 2   Low        757 non-null    float64
 3   Close      757 non-null    float64
 4   Adj Close  757 non-null    float64
 5   Volume     757 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 41.4 KB


In [27]:
#read news data
amazon = './newsarticles_amazon.csv'
df_amazon = pd.read_csv(amazon)
df_amazon.head()

,Date,Summary,Title,Article,Link
0,01-02-2019,Soon after Amazon (AMZN -5.60%) acquired Whole...,Amazon Will Take the Next Step of Its Whole Fo...,Check out the latest Amazon earnings call tran...,https://www.fool.com/investing/2019/01/02/amaz...
1,01-02-2019,Please Sign In and use this article's on page ...,Pepco alum Debbi Jarvis looks to boost Howard ...,Please Sign In and use this article's on page ...,https://www.bizjournals.com/washington/news/20...
2,01-02-2019,NaN,NaN,NaN,https://www.nasdaq.com/articles/why-prana-biot...
3,01-02-2019,NaN,NaN,NaN,https://www.nasdaq.com/articles/3-dividend-ari...
4,01-02-2019,"Over the past year, the shares of Starbucks Co...",Starbucks At Historic Low (NASDAQ:SBUX),"Over the past year, the shares of Starbucks Co...",https://seekingalpha.com/article/4231086-starb...


In [28]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4911 entries, 0 to 4910
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     4911 non-null   object
 1   Summary  4655 non-null   object
 2   Title    4662 non-null   object
 3   Article  4656 non-null   object
 4   Link     4911 non-null   object
dtypes: object(5)
memory usage: 192.0+ KB


In [62]:
#there are small portion of null values due to subscription issue. 
#remove rows with null values
df = df_amazon.dropna()
#remove rows with invalid data by check if 'amazon' or 'AMZN' appears in headline or summary
df = df[(df['Summary'].str.lower().str.contains('amazon'))|(df['Summary'].str.lower().str.contains('amzn'))|\
    (df['Title'].str.lower().str.contains('amazon'))|(df['Title'].str.lower().str.contains('amzn'))]

In [61]:

#combine title and article, drop link

,Date,Summary,Title,Article,Link
0,01-02-2019,Soon after Amazon (AMZN -5.60%) acquired Whole...,Amazon Will Take the Next Step of Its Whole Fo...,Check out the latest Amazon earnings call tran...,https://www.fool.com/investing/2019/01/02/amaz...
5,01-03-2019,What happenedShares of Amazon (AMZN -5.60%) we...,Why Amazon Stock Lost 11% Last Month,What happened\n\nShares of Amazon (AMZN -5.60%...,https://www.fool.com/investing/2019/01/03/why-...
8,01-04-2019,"As we approach the end of 2018, I take a look ...",Top 5 Tech Trends For 2019,"As we approach the end of 2018, I take a look ...",https://seekingalpha.com/article/4231443-top-5...
11,01-04-2019,Following a healthy eating plan can be challen...,Whole Foods online search tool filters food ch...,Following a healthy eating plan can be challen...,https://www.bizjournals.com/bizwomen/news/late...
13,01-05-2019,JPMorgan Chase & Co. Chief Executive James Dim...,Jeff Bezos and Jamie Dimon: Best of Frenemies,JPMorgan Chase & Co. Chief Executive James Dim...,https://www.wsj.com/articles/jeff-bezos-and-ja...
...,...,...,...,...,...
4902,12-30-2021,"In this Fool Live video clip, recorded on Dec....",Is Amazon a Good Stock to Buy For 2022?,Amazon (AMZN -5.45%) has been one of the best ...,https://www.fool.com/investing/2021/12/30/is-a...
4903,12-30-2021,(Bloomberg) -- A new California law designed t...,Amazon’s Machine Bosses Are Targeted in Califo...,(Bloomberg) -- A new California law designed t...,https://finance.yahoo.com/news/amazon-machine-...
4904,12-30-2021,2022 Could Be Banner Year for Amazon.comThese ...,2022 Could Be Banner Year for Amazon.com,2022 Could Be Banner Year for Amazon.com\n\nTh...,https://www.barrons.com/articles/2022-could-be...
4905,12-30-2021,Here's how to trade the stock as we gear up fo...,Will 2022 Be the Year for Amazon Stock to Hit ...,Amazon is up just 2.5% in 2021 with one tradin...,https://www.thestreet.com/investing/trading-am...


In [ ]:
import pandas

headlines_df = pandas.read_csv('c1a8c0359fbde2f6dcb92065b8ffc5e3/300_stock_headlines.csv')
headlines_df.head(5)

,Unnamed: 0,Unnamed: 0.1,headline,url,publisher,date,stock
0,0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [ ]:
import numpy as np

headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_list = list(headlines_array[:,2])

print(headlines_list)

['TDAM USA Inc. Buys Phillips 66, DuPont de Nemours Inc, Agilent Technologies Inc, Sells Apple ...', 'A, CSCO, NTAP and WPG among after hour movers', "Napier Park Global Capital LLC Buys Invesco Senior Loan, Agilent Technologies Inc, Lowe's Inc, ...", 'Stifel likes Illumina in premarket analyst action', "What's in the Offing for TripAdvisor's (TRIP) Q1 Earnings?", 'Agilent Technologies, Inc. (A) Management Presents at Evercore ISI HealthCONx Conference (Transcript)', 'BlueSpruce Investments, LP Buys Ross Stores Inc, TJX Inc, Comcast Corp, Sells Constellation ...', 'Stocks To Watch: Walmart And Alibaba Step To The Plate', "Square (SQ) to Report Q1 Earnings: What's in the Offing?", '21 Upcoming Dividend Increases To Round Out 2018', 'Royal London Asset Management Ltd Buys Apple Inc, Amazon. ...', 'Agilent Q3 2019 Earnings Preview', 'Stocks With Rising Relative Strength: Agilent Technologies', 'Agilent Technologies, Inc. 2019 Q4 - Results - Earnings Call Presentation', 'Stock Futures Plun

In [ ]:
!pip install transformers

#Getting the tokenizer and the model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

     |████████████████████████████████| 2.9 MB 2.0 MB/s 
     |████████████████████████████████| 636 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 55.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.9 MB/s 
     |████████████████████████████████| 52 kB 2.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
#That's where the headlines get tokenized to be inputted into model

inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[  101, 14595,  3286,  ...,     0,     0,     0],
        [  101,  1037,  1010,  ...,     0,     0,     0],
        [  101, 21320,  2380,  ...,     0,     0,     0],
        ...,
        [  101, 29003,  3372,  ...,     0,     0,     0],
        [  101, 29003,  3372,  ...,     0,     0,     0],
        [  101, 29003,  3372,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
#inference

outputs = model(**inputs)
print(outputs.logits.shape)

torch.Size([300, 3])


In [ ]:
#Postprocessing with softmax

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[0.0964, 0.0160, 0.8876],
        [0.0415, 0.0191, 0.9393],
        [0.0849, 0.0136, 0.9015],
        [0.1050, 0.0154, 0.8796],
        [0.0301, 0.2596, 0.7104],
        [0.0500, 0.0232, 0.9268],
        [0.0861, 0.0104, 0.9035],
        [0.1573, 0.0145, 0.8282],
        [0.0244, 0.4719, 0.5037],
        [0.3373, 0.0271, 0.6356],
        [0.1070, 0.0156, 0.8774],
        [0.3582, 0.0639, 0.5779],
        [0.8495, 0.0150, 0.1355],
        [0.0293, 0.0562, 0.9145],
        [0.0120, 0.9527, 0.0354],
        [0.1152, 0.0142, 0.8706],
        [0.1175, 0.0404, 0.8421],
        [0.0702, 0.0160, 0.9138],
        [0.0983, 0.0122, 0.8895],
        [0.7944, 0.0078, 0.1978],
        [0.0284, 0.0274, 0.9442],
        [0.0918, 0.1666, 0.7415],
        [0.9438, 0.0381, 0.0181],
        [0.0744, 0.0115, 0.9141],
        [0.0388, 0.0993, 0.8619],
        [0.8817, 0.0675, 0.0509],
        [0.6302, 0.0173, 0.3525],
        [0.0527, 0.0258, 0.9214],
        [0.8158, 0.0283, 0.1559],
        [0.040

In [ ]:
#Model classes

model.config.id2label

{0: 'positive', 1: 'negative', 2: 'neutral'}

In [ ]:
#Formatting the results as a pandas data frame

import pandas as pd

#Headline #Positive #Negative #Neutral
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

df.head(5)


,Headline,Positive,Negative,Neutral
0,"TDAM USA Inc. Buys Phillips 66, DuPont de Nemo...",0.096404,0.015965,0.887631
1,"A, CSCO, NTAP and WPG among after hour movers",0.041545,0.019125,0.939330
2,Napier Park Global Capital LLC Buys Invesco Se...,0.084920,0.013624,0.901456
3,Stifel likes Illumina in premarket analyst action,0.105007,0.015381,0.879612
4,What's in the Offing for TripAdvisor's (TRIP) ...,0.030088,0.259559,0.710353


In [ ]:
#Logging the pandas table as a WandB Table
!pip install wandb
import wandb

wandb.init(project="FinBERT_Sentiment_Analysis_Project")
wandb.run.log({"Financial Sentiment Analysis Table" : wandb.Table(dataframe=df)})
wandb.run.finish()

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 139 kB 53.6 MB/s 
     |████████████████████████████████| 97 kB 8.6 MB/s 
     |████████████████████████████████| 180 kB 69.6 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=5815a38cd8d1981760d00c602cdc6b07430b44e64aec4e9f37068fd3b671aee9
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=330d3b3978ff5853164c11bec7ac45269cf4ab0ef803cffec0742be482ef12cd
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
